# Modules to be imported

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import overpy
import psycopg2
import statistics
#import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras import layers
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

# Webscraping from www.immoscout24.ch

In [2]:
# Create an empty array to store the scraped data
properties = []

# List of URLs to scrape

urls = [
    'https://www.immoscout24.ch/en/real-estate/rent/city-winterthur',
    'https://www.immoscout24.ch/en/real-estate/rent/city-zuerich',
    'https://www.immoscout24.ch/en/real-estate/rent/city-geneve',
    'https://www.immoscout24.ch/en/real-estate/rent/city-basel',
    'https://www.immoscout24.ch/en/real-estate/rent/city-lausanne',
    'https://www.immoscout24.ch/en/real-estate/rent/city-bern',
    'https://www.immoscout24.ch/en/real-estate/rent/city-luzern',
    'https://www.immoscout24.ch/en/real-estate/rent/city-st-gallen',
    'https://www.immoscout24.ch/en/real-estate/rent/city-lugano',
    'https://www.immoscout24.ch/en/real-estate/rent/city-biel-bienne'
]   


In [3]:
# loop thorugh each URL 
def extract(link):
        L = []
        # Create a new instance of Chrome driver
        driver = webdriver.Chrome()

        # Navigate to the search result page for ten selected cities on ImmoScout24.ch
        driver.get(link)
        

        # Wait for the page to load completely
        driver.implicitly_wait(10)

        # Use BeautifulSoup to parse the HTML content of the page
        soup = BeautifulSoup(driver.page_source, "html.parser")

        if(link.endswith('thur')):
                pages = 5
        else:
                pages = soup.find("div",{"class": "Box-cYFBPY cJDTre"}).find_next().text

        print("this is the amount of pages: ", pages)

        for page in range(1,int(pages)+1):
        
            
            driver.get(link+'?pn='+str(page))

            # Find all the property listings on the page
            listings = soup.find_all("a", {"class": "Wrapper__A-kVOWTT"})


        # Loop through the listings and extract the required information
            for listing in listings:
                #print(listing)
                # Get the price, size, and address of the property
                price = listing.find("h3", {"class": re.compile("^Box-cYFBPY hKJGPR Heading")}).text.strip()
                #size = listing.find("h3", {"class": re.compile("^Box-cYFBPY hKJGPR Heading")}).text.strip()
                address = listing.find("span", {"class": "AddressLine__TextStyled-eaUAMD"}).text.strip()

                # Add the property details to the properties array as a tuple
                L.append((price, address))
                #print(properties)
        
        # Close the Chrome driver
        driver.close()
        return L
        # Print the scraped data array
        #print("Properties:", properties)

In [13]:
import multiprocess as mp
import time
from multiprocessing import Process, Manager
from itertools import repeat
from functools import partial
import concurrent.futures

start = time.perf_counter()


#with mp.Pool() as pool:
#    result = pool.map(extract,urls)
#    print(result)

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor: 
    result = executor.map(extract,  urls)
    
finish = time.perf_counter()

print(f'Finished urls in {round(finish-start,2)} second(s)')

this is the amount of pages:  22
this is the amount of pages:  40
this is the amount of pages:  25
this is the amount of pages:  25
this is the amount of pages:  5
this is the amount of pages:  21
this is the amount of pages:  6
this is the amount of pages:  28
this is the amount of pages:  13
this is the amount of pages:  21
Finished urls in 3921.86 second(s)


In [14]:
r = list(result)

In [15]:
r

[[('3.5 rooms, 83 m², CHF 1920.—', 'Hegistr. 35d, 8404 Winterthur, ZH'),
  ('2.5 rooms, 55 m², CHF 1770.—',
   'Wülflingerstrasse 155, 8408 Winterthur, ZH'),
  ('4.5 rooms, 105 m², CHF 1873.—',
   'Landvogt-Waser-Strasse 33, 8405 Winterthur, ZH'),
  ('1 room, 26 m², CHF 1195.—', 'Wartstr. 28, 8400 Winterthur, ZH'),
  ('4.5 rooms, 171 m², CHF 3339.—',
   'Stadthausstrasse 12, 8400 Winterthur, ZH'),
  ('6.5 rooms, 200 m², CHF 4450.—',
   'Schlosshofstrasse 218, 8404 Winterthur, ZH'),
  ('3 rooms, 70 m², CHF 2010.—', 'Schaffhauserstrasse 6, 8400 Winterthur, ZH'),
  ('2.5 rooms, 56 m², CHF 1740.—', 'Stadthausstr. 11, 8400 Winterthur, ZH'),
  ('3.5 rooms, 96 m², CHF 2230.—',
   'Else-Züblin-Strasse 73, 8404 Winterthur, ZH'),
  ('2.5 rooms, 70 m², CHF 2270.—',
   'jonas-furrer-strasse 90, 8400 Winterthur, ZH'),
  ('2.5 rooms, 63 m², CHF 2335.—', 'Obertor 23 / 23a, 8400 Winterthur, ZH'),
  ('5.5 rooms, 143 m², CHF 3410.—',
   'Else-Züblin-Str. 72, 8404 Winterthur, ZH'),
  ('4.5 rooms, 121 m²,

In [18]:
import csv
with open('immo', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerows(r)

In [16]:
df = pd.DataFrame(r)

In [17]:
df

,0,1,2,3,4,5,6,7,8,9,...,950,951,952,953,954,955,956,957,958,959
0,"(3.5 rooms, 83 m², CHF 1920.—, Hegistr. 35d, 8...","(2.5 rooms, 55 m², CHF 1770.—, Wülflingerstras...","(4.5 rooms, 105 m², CHF 1873.—, Landvogt-Waser...","(1 room, 26 m², CHF 1195.—, Wartstr. 28, 8400 ...","(4.5 rooms, 171 m², CHF 3339.—, Stadthausstras...","(6.5 rooms, 200 m², CHF 4450.—, Schlosshofstra...","(3 rooms, 70 m², CHF 2010.—, Schaffhauserstras...","(2.5 rooms, 56 m², CHF 1740.—, Stadthausstr. 1...","(3.5 rooms, 96 m², CHF 2230.—, Else-Züblin-Str...","(2.5 rooms, 70 m², CHF 2270.—, jonas-furrer-st...",...,None,None,None,None,None,None,None,None,None,None
1,"(3.5 rooms, 69 m², CHF 3750.—, Nelkenstrasse 2...","(1.5 rooms, 23 m², CHF 1460.—, Viktoriastrasse...","(1.5 rooms, 26 m², CHF 1820.—, Eugen-Huber-Str...","(4 rooms, 69 m², CHF 1444.—, Dübendorfstrasse ...","(2.5 rooms, 81 m², CHF 2260.—, Heerenschürlist...","(2.5 rooms, 106 m², CHF 3380.—, Freischützgass...","(2 rooms, 57 m², CHF 1650.—, Röslistr. 57, 800...","(2.5 rooms, 62 m², CHF 2080.—, Maneggstrasse 8...","(4 rooms, 110 m², CHF 3390.—, Toblerstrasse 10...","(4.5 rooms, 83 m², CHF 3965.—, Hildastrasse, 8...",...,None,None,None,None,None,None,None,None,None,None
2,"(7.5 rooms, 161 m², CHF 6510.—, Boulevard des ...","(4 rooms, 63 m², CHF 3180.—, Rue du 31-Décembr...","(4 rooms, 90 m², CHF 5410.—, Rue du Rhône 57, ...","(3 rooms, 58 m², CHF 2180.—, Louis-Curval 3, 1...","(2.5 rooms, CHF 2550.—, Route de Florissant 89...","(5 rooms, CHF 3505.—, chemin Rieu, 5, 1208 Gen...","(4 rooms, 67 m², CHF 2705.—, Place du Cirque 4...","(3 rooms, 57 m², CHF 2579.—, Rte de Frontenex ...","(3 rooms, 60 m², CHF 2627.—, Rte de Frontenex ...","(80 m², Price on request, Quai du Mont-Blanc 2...",...,None,None,None,None,None,None,None,None,None,None
3,"(3.5 rooms, 84 m², CHF 1790.—, Burgfelderstras...","(2.5 rooms, 52 m², CHF 2200.—, Erlenmattstrass...","(1.5 rooms, 37 m², CHF 2000.—, Gerbergasse 40,...","(2.5 rooms, 99 m², CHF 2250.—, Claragraben 78,...","(5 rooms, 145 m², CHF 3510.—, Greifengasse 19,...","(3.5 rooms, 97 m², CHF 2880.—, Magnolienpark 1...","(4 rooms, 86 m², CHF 1940.—, St. Alban-Ring 22...","(3 rooms, 72 m², CHF 1825.—, Münchensteinerstr...","(2.5 rooms, 60 m², CHF 1650.—, Erlenmattstr. 7...","(3 rooms, 79 m², CHF 1750.—, Sevogelstr. 40, 4...",...,"(3 rooms, 90 m², CHF 2100.—, Colmarerstrasse 2...","(2.5 rooms, 59 m², CHF 1340.—, Gilgenbergerstr...","(1 room, 37 m², CHF 1050.—, St. Alban-Rheinweg...","(3 rooms, 67 m², CHF 1610.—, Vogesenstrasse 31...","(4 rooms, 140 m², CHF 3150.—, Elsässerstrasse ...","(3 rooms, 68 m², CHF 1790.—, St. Alban-Vorstad...","(1 room, 45 m², CHF 1490.—, Gerbergasse 16, 40...","(4.5 rooms, 105 m², CHF 2900.—, Wanderstrasse ...","(6.5 rooms, 140 m², CHF 3400.—, Wettsteinplatz...","(3.5 rooms, 72 m², CHF 1930.—, 190 Klybeckstra..."
4,"(2.5 rooms, 45 m², CHF 1460.—, Rue du Petit-Ro...","(1 room, 27 m², CHF 1010.—, Avenue d' Echallen...","(2.5 rooms, 67 m², CHF 2050.—, Rue Edith-Burge...","(3.5 rooms, 84 m², CHF 2230.—, Rue Edith-Burge...","(4.5 rooms, 105 m², CHF 2400.—, Rue Edith-Burg...","(3 rooms, 74 m², CHF 1690.—, Avenue de la Vall...","(3 rooms, 77 m², CHF 2210.—, Avenue Juste Oliv...","(4.5 rooms, 107 m², CHF 2605.—, Av. des Bovere...","(2 rooms, 102 m², CHF 2430.—, Rue du Grand-Pon...","(3 rooms, 67 m², CHF 1950.—, Avenue Marc Dufou...",...,None,None,None,None,None,None,None,None,None,None
5,"(2.5 rooms, 62 m², CHF 1890.—, Seftigenstrasse...","(3 rooms, 67 m², CHF 1930.—, Weissensteinstras...","(3.5 rooms, 75 m², CHF 2910.—, Zeughausgasse 2...","(2.5 rooms, 105 m², CHF 1860.—, Sickingerstras...","(4.5 rooms, 110 m², CHF 2050.—, Brünnenstrasse...","(3 rooms, 74 m², CHF 2500.—, Marzilistrasse 38...","(1.5 rooms, 42 m², CHF 1480.—, Muristrasse 64,...","(1.5 rooms, 40 m², CHF 1440.—, Muristrasse 64,...","(4.5 rooms, CHF 3780.—, Adresse auf Anfrage, 3...","(5.5 rooms, 146 m², CHF 6100.—, 3000 Bern, BE)",...,None,None,None,None,None,None,None,None,None,None
6,"(3 rooms,

# WEB API using Overpass Turbo

In [7]:
# defining the overpass query
query = '''
[out:json];
{{geocodeArea:Zurich}} -> .searchArea0;
{{geocodeArea:Geneva}} -> .searchArea1;
{{geocodeArea:Basel}} -> .searchArea2;
{{geocodeArea:Lausanne}} -> .searchArea3;
{{geocodeArea:Bern}} -> .searchArea4;
{{geocodeArea:Winterthur}} -> .searchArea5;
{{geocodeArea:Luzern}} -> .searchArea6;
{{geocodeArea:St. Gallen}} -> .searchArea7;
{{geocodeArea:Lugano}} -> .searchArea8;
{{geocodeArea:Biel}} -> .searchArea9;
( area.searchArea0; area.searchArea1; area.searchArea2; area.searchArea3; area.searchArea4; area.searchArea5; area.searchArea6; area.searchArea7; area.searchArea8; area.searchArea9;) -> .searchArea;
(node["shop"="supermarket"](area.searchArea);way["shop"="supermarket"](area.searchArea););
out center;
'''

# creating Overpass API object
api = overpy.Overpass()

# performing the query and retrieveing the results
response = api.query(query)

# print the name and zip code of each supermarket
for element in response["elements"]:
    if element["tags"].get("name") and element["tags"].get("addr:postcode"):
        name = element["tags"]["name"]
        zip_code = element["tags"]["addr:postcode"]
        data = (name, zip_code)

OverpassBadRequest: Error: line 3: parse error: Unknown type &quot;{&quot; 
Error: line 3: parse error: An empty query is not allowed 
Error: line 3: parse error: ';' expected - '{' found. 
Error: line 4: parse error: Unknown type &quot;{&quot; 
Error: line 4: parse error: An empty query is not allowed 
Error: line 4: parse error: ';' expected - '{' found. 
Error: line 5: parse error: Unknown type &quot;{&quot; 
Error: line 5: parse error: An empty query is not allowed 
Error: line 5: parse error: ';' expected - '{' found. 
Error: line 6: parse error: Unknown type &quot;{&quot; 
Error: line 6: parse error: An empty query is not allowed 
Error: line 6: parse error: ';' expected - '{' found. 
Error: line 7: parse error: Unknown type &quot;{&quot; 
Error: line 7: parse error: An empty query is not allowed 
Error: line 7: parse error: ';' expected - '{' found. 
Error: line 8: parse error: Unknown type &quot;{&quot; 
Error: line 8: parse error: An empty query is not allowed 
Error: line 8: 

# Connection to AWS PostgreSQL and inserting Data

In [ ]:
#Connection to the database
conn = psycopg2.connect(
    host="ads-database1.cbwqb3cep5ch.eu-central-1.rds.amazonaws.com",
    database="adsdatabase",
    user="adschief1",
    password="1eYvmDnMPikKSImeLrev"
)

# Creating a cursor object for executing SQL statements
cur = conn.cursor()

# Iterate over the properties array
for property in properties:
    # Extract the property details
    details = property[0].split(", ")  # Split the details by comma and space

    # Extract the individual details
    rooms = details[0]
    size = details[1]
    price = details[2]
    address = details[3]
    zipcode = details[4]
    kanton = details [5]
    
    # Process the individual data record as needed
    print("Rooms:", rooms)
    print("Size:", size)
    print("Price:", price)
    print("Address:", address)
    print("Zip-Code", zipcode)
    print("Kanton", kanton)
    print("--------------------")


    # Insert the data from ImmoScout24 into the database
    cur.execute(
        "INSERT INTO your_table_name (rooms, size, price, address, zipcode, kanton) VALUES (%s, %s, %s, %s. %s, %s)",
        (rooms, size, price, address, zipcode, kanton)
    )
    
    # Insert the data from Overpass Turbo into the database
    cur.execute(
        "INSERT INTO your_table_name2 (name, zip_code) VALUES (%s, %s)"
        (name, zip_code)
    )

# Commit the changes to the database
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()


#### Join the two Tables "immoscout24_data" and "overpass_data"

In [ ]:
# Join the two tables 
query = '''
SELECT *
FROM immoscout24_data
JOIN overpass_data ON immoscout24_data.zip_code = overpass_data.zip_code
'''
data_df = pd.read_sql(query, conn)


# EDA using our data from PostgreSQL

In [ ]:
# First few rows of the DataFrame
data_df.head()

In [ ]:
# Summary statistics
data_df.describe()

In [ ]:
# Examine the data types
data_df.dtypes

In [ ]:
# Count number of missing values
data_df.isnull().sum()

In [ ]:
#mean, median, mean, median, mode, variance, standard deviation
#histogram, boxplots, scatter plots
#correlation analysis
#hypothesis testing 

# Retrieve the data from the database
cur.execute("SELECT price FROM XXXXXXXXXXXX")

# Fetch all the rows of the query result
rows = cur.fetchall()

# Extract the values from the rows
prices = [row[0] for row in rows]

# Calculate the mean
mean = statistics.mean(prices)

# Calculate the median
median = statistics.median(prices)

# Calculate the mode
mode = statistics.mode(prices)

# Calculate the variance
variance = statistics.variance(prices)

# Calculate the standard deviation
std_dev = statistics.stdev(prices)

# Print the results
print("Mean:", mean)
print("Median:", median)
print("Mode:", mode)
print("Variance:", variance)
print("Standard Deviation:", std_dev)

# Correlation Matrix 

### How do we separate the data into training and testing data? NO CLUE

In [ ]:
# Split the data into training and test sets (70% training, 30% testing)
train_df, test_df = train_test_split(data_df, test_size=0.3, random_state=42)


# Create the correlation matrix
correlation_matrix = train_df.corr()

print(correlation_matrix)

### Correlation Matrix Plot

In [ ]:
# Correlation Matrix Plot (corrplot)
sns.pairplot(train_df)